In [1]:
!pip install langgraph langchain langchain-groq langchain-community duckduckgo-search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os

os.environ["GROQ_API_KEY"] = "gsk_Hokn18tWuDKl5Jn*********************Gk9olc"


In [3]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

class GraphState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    route: str


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [5]:
from langchain_core.messages import HumanMessage

def router_agent(state: GraphState):
    last_message = state["messages"][-1].content

    prompt = f"""
You are a router.
Classify the user query into ONE category:

- general
- math
- search
- weather

Query: {last_message}

Reply with only one word.
"""

    response = llm.invoke(prompt).content.strip().lower()

    return {
        "route": response
    }


In [6]:
def general_agent(state: GraphState):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }


In [30]:
from langchain_core.tools import tool
import re

@tool
def math_tool(expression: str) -> str:
    """Evaluate a mathematical expression"""
    return str(eval(expression))


def math_agent(state: GraphState):
    query = state["messages"][-1].content

    # Find ALL valid math substrings
    matches = re.findall(r"[\d\.]+|[\+\-\*/\(\)]", query)

    if not matches:
        return {
            "messages": ["I could not find a valid mathematical expression."]
        }

    # Join tokens into a clean expression
    expression = "".join(matches)

    try:
        result = math_tool.invoke(expression)
    except Exception as e:
        return {
            "messages": [f"Math error: {str(e)}"]
        }

    return {
        "messages": [result]
    }


In [9]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.8 MB/s eta 0:00:00


In [10]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

def search_agent(state: GraphState):
    query = state["messages"][-1].content
    result = search_tool.invoke(query)
    return {
        "messages": [result]
    }


In [11]:
def weather_tool(city: str):
    return f"Today's weather in {city}: 30°C, Sunny"

def weather_agent(state: GraphState):
    query = state["messages"][-1].content
    return {
        "messages": [weather_tool(query)]
    }


In [12]:
def route_decision(state: GraphState):
    return state["route"]


In [31]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = StateGraph(GraphState)

graph.add_node("router", router_agent)
graph.add_node("general", general_agent)
graph.add_node("math", math_agent)
graph.add_node("search", search_agent)
graph.add_node("weather", weather_agent)

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",
    route_decision,
    {
        "general": "general",
        "math": "math",
        "search": "search",
        "weather": "weather",
    }
)

graph.add_edge("general", END)
graph.add_edge("math", END)
graph.add_edge("search", END)
graph.add_edge("weather", END)

app = graph.compile(checkpointer=memory)


In [32]:
app = graph.compile(checkpointer=memory)

In [33]:
config = {
    "configurable": {
        "thread_id": "user-1"
    }
}


In [34]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "user-1"}}

result = app.invoke(
    {
        "messages": [
            HumanMessage(content="What is 56 * 23?")
        ]
    },
    config=config
)

result["messages"][-1].content


'1288'

In [35]:
result = app.invoke(
    {
        "messages": [
            HumanMessage(content="Who is the CEO of OpenAI?")
        ]
    },
    config=config
)

result["messages"][-1].content


'Sam Altman is the CEO of OpenAI and one of the world\'s leading advocates for artificial intelligence. Read about his net worth, husband, sister, and more. OpenAI CEO : The Chief Executive Officer of OpenAI is a well-known technology entrepreneur and investor. This person oversees the company\'s overall strategy, research direction, partnerships, and ... OpenAI CEO Sam Altman became a household name after the release of OpenAI\'s groundbreaking AI model, ChatGPT. To strengthen our execution, I\'m excited to announce Fidji Simo is joining as our CEO of Applications, reporting directly to me. I remain the CEO of OpenAI and will continue to directly oversee success across all pillars of OpenAI - Research, Compute, and Applications - ensuring we stay aligned and integrated across all areas. Reports of OpenAI\'s plans to go public coincide with Altman declaring a "code red" for the company, something he said he plans to do multiple times in the foreseeable future.'

In [36]:
result = app.invoke(
    {
        "messages": [
            HumanMessage(content="Today's weather in Bangalore")
        ]
    },
    config=config
)

result["messages"][-1].content


"Today's weather in Today's weather in Bangalore: 30°C, Sunny"

In [37]:
result = app.invoke(
    {
        "messages": [
            HumanMessage(content="Explain agentic AI in simple words")
        ]
    },
    config=config
)

result["messages"][-1].content


"Agentic AI refers to a type of artificial intelligence that has the ability to make decisions and take actions on its own, similar to how humans act as agents in the world.\n\nImagine you're giving instructions to a robot to clean your room. A non-agentic AI would follow the instructions exactly, but an agentic AI would be able to understand the context, make decisions, and take actions that are not explicitly programmed.\n\nFor example, if the robot sees that the room is messy and there's a lot of dust, an agentic AI might decide to use a vacuum cleaner instead of a broom, or even suggest cleaning the room in a different way. It's like the AI is taking initiative and acting like a human agent would.\n\nAgentic AI is still a developing area of research, but it has the potential to revolutionize many areas, such as:\n\n- Robotics: Agentic AI could enable robots to perform complex tasks and adapt to new situations.\n- Healthcare: Agentic AI could help doctors and nurses make decisions a

In [38]:
result = app.invoke(
    {
        "messages": [
            HumanMessage(content="Remember that my name is Sayandeep")
        ]
    },
    config=config
)

result["messages"][-1].content


"Nice to remember that, Sayandeep. I'll keep that in mind for our conversation. How can I assist you further?"

In [39]:
result = app.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ]
    },
    config=config
)

result["messages"][-1].content


'Your name is Sayandeep.'

In [40]:
config_user2 = {
    "configurable": {
        "thread_id": "user-2"
    }
}

result = app.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ]
    },
    config=config_user2
)

result["messages"][-1].content


"I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users or their personal details. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!"